In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

sys.path.append('..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from utils import sparsify_beta
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

from error_bounds import calc_error_bounds

from pyuoi.linear_model import UoI_Lasso

First, on a smaller scale, verify: 
1) Invariance to choice of subset
2) Correspondence between our bounds and MC OMLE decoder

Other thing to check: There is a free parameter in our decomposition (how much of the diagonal to assign to the uncorrelated portion). Do our results depend on this parameter?

Generate covariance matrices and then: 

1) Compute their latent variable decomposition
2) Compute the error probability
3) Compute the necessary sample size

In [3]:
# Beta has to be fixed
beta = gen_beta2(100, 100, sparsity = 1, betawidth = np.inf, seed = 1234)
beta = sparsify_beta(beta, 100, 0.2, seed = 1234)

# Generate a fixed covariance matrix. For now, we will just vary SNR 
sigma = gen_covariance(100, 0, 100, 5, 1)

In [10]:
from error_bounds import sigma_decomposition

In [12]:
L, _ = sigma_decomposition(sigma)

In [20]:
np.linalg.det(np.linalg.pinv(L))

-1.503941057800979e+20

In [18]:
np.prod(np.abs(np.linalg.eig(L)[0][1:]))

5.868893389571986e-53

In [ ]:
np.product(np.eig(L)[0])

In [ ]:
kappa = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 250, 500, 1000]
bounds = np.zeros(len(kappa))
for i, k in enumerate(kappa):
    # Calculate information theoretic bounds on error probability and number of samples needed
    bounds[i] = calc_error_bounds(sigma, k, beta.ravel())

> /home/akumar/nse/uoicorr/infotheory/error_bounds.py(179)calc_errorexpon()
-> return delta, E0
(Pdb) delta.shape
torch.Size([])
(Pdb) delta
tensor(0., requires_grad=True)
(Pdb) E0.shape
torch.Size([19, 19])
(Pdb) torch.log(alpha0 * sigma**2)
tensor(-14.5617, grad_fn=<LogBackward>)
(Pdb) torch.log(torch.det(Linv))
tensor(inf)
(Pdb) Linv
tensor([[-0.0067, -0.0080, -0.0092,  ..., -0.0092, -0.0080, -0.0067],
        [ 0.0136,  0.0160,  0.0183,  ..., -0.0183, -0.0160, -0.0136],
        [-0.0205, -0.0240, -0.0274,  ..., -0.0274, -0.0240, -0.0205],
        ...,
        [-0.5100,  1.4295, -2.3266,  ..., -2.3266,  1.4295, -0.5100],
        [ 0.4929, -1.3503,  2.1007,  ..., -2.1007,  1.3503, -0.4929],
        [ 0.4969, -1.3767,  2.1904,  ..., -2.1904,  1.3767, -0.4969]])
(Pdb) torch.det(Linv)
tensor(inf)
(Pdb) torch.log(torch.det(G))
tensor(inf, grad_fn=<LogBackward>)
(Pdb) G
tensor([[ 100.8442,    5.9890,   14.2652,   13.1659,   15.1801,    3.0248,
            9.2150,    8.2553,   16.2833,   1

In [ ]:
# Will need to make beta smaller